<a href="https://colab.research.google.com/github/slbryson/MLlearn/blob/master/Para_WhisperGPUTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
import multiprocessing

def benchmark_model(model, input_tensor, device, num_iterations):
    # Move the model to the current device
    model.to(device)

    # Move the input tensor to the current device
    input_tensor = input_tensor.to(device)

    # Run the model once to initialize the GPU
    _ = model.transcribe(input_tensor)

    # Benchmark the GPU by running the model multiple times
    times = []
    for i in range(num_iterations):
        start_time = time.time()
        _ = model.transcribe(input_tensor)
        end_time = time.time()
        times.append(end_time - start_time)

    # Calculate the average time per iteration
    avg_time_per_iteration = sum(times) / num_iterations

    # Print the average time per iteration
    print(f"Average time per iteration: {avg_time_per_iteration:.5f} seconds")

    # Return the average time per iteration
    return avg_time_per_iteration


In [2]:

# Check if CUDA is available
if torch.cuda.is_available():
    # Use the first available GPU device
    device = torch.device("cuda:0")
    # Print the current device
    print(f"Using device: {torch.cuda.get_device_name(device)}")
else:
    print("CUDA is not available, using CPU instead.")
    device = torch.device("cpu")


Using device: Tesla T4


In [3]:
#Grab the whisper library
!pip install -U openai-whisper
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 16.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230124-py3-none-any.whl size=1179329 sha256=55f0b9a3256a60074ffc3c294ea33e584b55c6c9f3571d1acf2017538bd8aa19
  Stored in directory: /root/.cache/pip/wheels/48/c2/dd/8639c7cda1e20412e499ab65e5003d8863ef8622792ea26446
Successfully built openai-whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ddxv456o
  Running command git clone --filter=blob:none --qui

Load the model

In [6]:

import whisper

# Load the large model
model_type = 'medium'
model1 = whisper.load_model(model_type)
model2 = whisper.load_model(model_type)


100%|██████████████████████████████████████| 1.42G/1.42G [00:06<00:00, 221MiB/s]


In [13]:

# Set the Length of the audio
audio_len = 5
# Generate a random audio signal of length 1 second and sampling rate 44100 Hz
audio_signal = np.random.randn(44100*audio_len).astype(np.float32)

# Write the audio signal to a WAV file
write('random_audio.wav', 44100, audio_signal)

# Load the audio file using whisper
input_tensor = whisper.load_audio('random_audio.wav')
input_tensor = whisper.pad_or_trim(input_tensor)
# Reshape the input tensor to have shape (1, num_samples, num_channels)
# input_tensor = input_tensor.reshape((1, -1, 1))

# Move the input tensor to the current device
input_tensor = torch.from_numpy(input_tensor).to(device)




In [17]:
# Check the Input
# Run the model once to initialize the GPU
# _ = model.transcribe(input_tensor)

result = model1.transcribe(input_tensor)
print("Results from Model 1")
print(result["text"])


Results from Model 1
 Thanks for watching!


In [18]:
result = model2.transcribe(input_tensor)
print(result["text"])

 Thanks for watching!


In [20]:
avg_time = benchmark_model(model2,input_tensor,device,2)

Average time per iteration: 0.65331 seconds


In [32]:
inputs= [model1,input_tensor,device,2]
avg1 =[]
with multiprocessing.Pool(processes=2) as pool:
  avg1 = pool.imap(benchmark_model,inputs)

print(avg1)

In [ ]:

# Number of Iterations
num_iter = 1
# Create two processes to run the models in parallel
process1 = multiprocessing.get_context("spawn").Process(target=benchmark_model, args=(model1, input_tensor, device, num_iter))
process2 = multiprocessing.get_context("spawn").Process(target=benchmark_model, args=(model2, input_tensor, device, num_iter))

# Start the processes
process1.start()
process2.start()

# Wait for the processes to finish
process1.join()
process2.join()
